# 互评作业2：频繁模型与关联规则挖掘

**姓名**：杨毅哲

**学号**：3120201088

**GitHub**：https://github.com/YangPatrick/DataMining

## 包引用


In [1]:
import pandas as pd
import collections
import itertools
import typing

## 数据集处理

对数据集进行处理，转换成适合进行关联规则挖掘的形式。将对该实验所用的Wine Reviews数据集进行倒入、清理并进行可视化，为之后的频繁模式和关联规则的挖掘做准备。

数据加载，并对该数据集中行、列进行统计，并简要进行可视化

In [2]:
wine_review_dataset = pd.read_csv('winemag-data-130k-v2.csv')
wine_review_dataset.info()
wine_review_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


根据每列属性的含义，我们选择country、designation、province、variety和winery等几个属性进行数据清洗，由于存在缺失值，我们将这些缺失值用nan进行填充，并在此基础上进行关联规则挖掘。

In [3]:
columns = ['country', 'designation', 'province', 'taster_name', 'variety', 'winery']
wine_review_dataset = wine_review_dataset[columns].fillna('nan')
data = wine_review_dataset.values.tolist()

## 找出频繁模式

寻找频繁模式并统计频繁模式的信息，为了在特定的数据和指定的支持度下进行频繁模式的查找，定义如下函数，查找支持度达到0.05的频繁模式。

In [4]:
def apriori(itemsets):
    i = 0
    possible_itemsets = []
    while i < len(itemsets):
        skip = 1
        *first, last = itemsets[i]
        tail = [last]
        for j in range(i + 1, len(itemsets)):
            *n_first, n_last = itemsets[j]
            if first == n_first:
                tail.append(n_last)
                skip += 1
            else:
                break
        first = tuple(first)
        for a, b in sorted(itertools.combinations(tail, 2)):
            possible_itemsets.append(first + (a,) + (b,))
        i += skip
    itemsets = set(itemsets)
    for possible_itemset in possible_itemsets:
        for i in range(len(possible_itemset) - 2):
            removed = possible_itemset[:i] + possible_itemset[i+1 :]
            if removed not in itemsets:
                break
        yield possible_itemset
    


def find_frequent_patterns(data, min_support, max_length):
    data_sets = [set(item) for item in data if len(item) > 0]
    data = data_sets
    use_data = collections.defaultdict(lambda: True)
    
    counts = collections.defaultdict(int)
    num_data = len(data_sets)
    for d in data:
        for item in d:
            counts[item] += 1
            
    large_itemsets = [
        (item, count)
        for (item, count) in counts.items()
        if (count / num_data) >= min_support
    ]
    
    if len(large_itemsets) < 0:
        return dict(), num_data
    else:
        large_itemsets = {1: {(item, ): count for (item, count) in sorted(large_itemsets)}}
    
    k = 2
    while large_itemsets[k - 1] and max_length != 1:
        itemsets_list = list(large_itemsets[k-1].keys())
        C_k = list(apriori(itemsets_list))
        C_k_sets = [set(itemset) for itemset in C_k]
        
        if not C_k:
            break
        
        candidate_itemset_counts = collections.defaultdict(int)
        for row, d in enumerate(data):
            if not use_data[row]:
                continue
            found_any = False
            for candidate, candidate_set in zip(C_k, C_k_sets):
                if candidate_set.issubset(d):
                    candidate_itemset_counts[candidate] += 1
                    found_any = True
            if not found_any:
                use_data[row] = False
        
        C_k = [
            (item, count)
            for (item, count) in candidate_itemset_counts.items()
            if (count / num_data) >= min_support
        ]
        
        if not C_k:
            break
        
        large_itemsets[k] = {item: count for (item, count) in sorted(C_k)}
        
        k += 1
        if k > max_length:
            break
            
    return large_itemsets, num_data

itemsets, num_data = find_frequent_patterns(data=data, min_support=0.05, max_length=8)

展示所有找到的频繁模式：

In [5]:
for idx, itemset in itemsets.items():
    print(f'idx-itemset:')
    for items in itemset:
        print(items)
    print('\n')

idx-itemset:
('Bordeaux-style Red Blend',)
('Cabernet Sauvignon',)
('California',)
('Chardonnay',)
('France',)
('Italy',)
('Kerin O’Keefe',)
('Michael Schachner',)
('Paul Gregutt',)
('Pinot Noir',)
('Red Blend',)
('Roger Voss',)
('Spain',)
('US',)
('Virginie Boone',)
('Washington',)
('nan',)


idx-itemset:
('Cabernet Sauvignon', 'US')
('California', 'Pinot Noir')
('California', 'US')
('California', 'Virginie Boone')
('California', 'nan')
('Chardonnay', 'US')
('France', 'Roger Voss')
('France', 'nan')
('Italy', 'Kerin O’Keefe')
('Italy', 'nan')
('Michael Schachner', 'Spain')
('Paul Gregutt', 'US')
('Pinot Noir', 'US')
('Roger Voss', 'nan')
('US', 'Virginie Boone')
('US', 'Washington')
('US', 'nan')


idx-itemset:
('California', 'Pinot Noir', 'US')
('California', 'US', 'Virginie Boone')
('California', 'US', 'nan')
('France', 'Roger Voss', 'nan')




## 导出关联规则并计算支持度和置信度

我们先定义一个关联规则类用于描述规则

In [6]:
class Rule:
    _decimals = 3
    
    def __init__(
        self,
        lhs: tuple,
        rhs: tuple,
        count_full: int = 0,
        count_lhs: int = 0,
        count_rhs: int = 0,
        num_datas: int = 0
    ):
        self.lhs = lhs
        self.rhs = rhs
        self.count_full = count_full
        self.count_lhs = count_lhs
        self.count_rhs = count_rhs
        self.num_data = num_data
        
    @property
    def confidence(self):
        try:
            return self.count_full / self.count_lhs
        except ZeroDivisionError:
            return None
    
    @property
    def support(self):
        try:
            return self.count_full / self.num_data
        except ZeroDivisionError:
            return None
        
    @staticmethod
    def _pf(s):
        return '{' + ', '.join(str(k) for k in s) + '}'
    
    def __str__(self):
        return f'{self._pf(self.lhs)} -> {self._pf(self.rhs)}'
    
    def __eq__(self, another_rule):
        return (set(self.lhs) == set(another_rule.lhs)) and (set(self.rhs) == set(another_rule.rhs))
    
    def __hash__(self):
        return hash(frozenset(self.lhs + self.rhs))
    
    def __len__(self):
        return len(self.lhs + self.rhs)

定义函数，挖掘关联规则

In [7]:
def generate_rules_apriori(
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_confidence: float,
    num_transactions: int,
    verbosity: int = 0,
):
    def count(itemset):
        return itemsets[len(itemset)][itemset]

    for size in itemsets.keys():
        if size < 2:
            continue
        if verbosity > 0:
            print(" Generating rules of size {}.".format(size))

        for itemset in itemsets[size].keys():
            for removed in itertools.combinations(itemset, 1):
                lhs = set(itemset).difference(set(removed))
                lhs = tuple(sorted(list(lhs)))
                conf = count(itemset) / count(lhs)
                if conf >= min_confidence:
                    yield Rule(
                        lhs,
                        removed,
                        count(itemset),
                        count(lhs),
                        count(removed),
                        num_transactions,
                    )

            H_1 = list(itertools.combinations(itemset, 1))
            yield from _ap_genrules(
                itemset, H_1, itemsets, min_confidence, num_transactions
            )



def _ap_genrules(
    itemset: tuple,
    H_m: typing.List[tuple],
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_conf: float,
    num_transactions: int,
):

    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if len(itemset) <= (len(H_m[0]) + 1):
        return

    H_m = list(apriori(H_m))
    H_m_copy = H_m.copy()

    for h_m in H_m:
        lhs = tuple(sorted(list(set(itemset).difference(set(h_m)))))
        if (count(itemset) / count(lhs)) >= min_conf:
            yield Rule(
                lhs,
                h_m,
                count(itemset),
                count(lhs),
                count(h_m),
                num_transactions,
            )
        else:
            H_m_copy.remove(h_m)
    if H_m_copy:
        yield from _ap_genrules(
            itemset, H_m_copy, itemsets, min_conf, num_transactions
        )

In [8]:
rules = generate_rules_apriori(
    itemsets=itemsets, 
    min_confidence=0.1, 
    num_transactions=num_data, 
    verbosity=0,
)
rules = list(rules)

计算每个规则对支持度和置信度，并对关联规则和它们的属性进行可视化展示

In [9]:
rules_temp = []
for rule in rules:
    valid = True
    for val in rule.lhs + rule.rhs:
        if 'nan' in val:
            valid = False
            break
    if valid:
        rules_temp.append(rule)
rules = rules_temp

rules_df = pd.DataFrame(
    {
        'Rules': rules,
        'Support': list(map(lambda x: x.support, rules)),
        'Confidence': list(map(lambda x: x.confidence, rules))
    }
)

rules_df

,Rules,Support,Confidence
0,{US} -> {Cabernet Sauvignon},0.056289,0.134229
1,{Cabernet Sauvignon} -> {US},0.056289,0.771730
2,{Pinot Noir} -> {California},0.053081,0.518644
3,{California} -> {Pinot Noir},0.053081,0.190333
4,{US} -> {California},0.278885,0.665034
5,{California} -> {US},0.278885,1.000000
6,{Virginie Boone} -> {California},0.073339,0.999476
7,{California} -> {Virginie Boone},0.073339,0.262973
8,{US} -> {Chardonnay},0.052350,0.124835
9,{Chardonnay} -> {US},0.052350,0.578768


## 评价关联规则
### Lift 指标
定义Lift指标函数如下：

In [10]:
def lift(rule):
    observed_support = rule.count_full / rule.num_data
    expected_support = (rule.count_lhs * rule.count_rhs) / rule.num_data ** 2
    return observed_support / expected_support

### Conviction 指标
Conviction指标定义函数如下：

In [11]:
def conviction(rule):
    eps = 1e-6
    prob_not_rhs = 1 - rule.count_rhs / rule.num_data
    prob_not_rhs_given_lhs = 1- rule.confidence
    return prob_not_rhs / (prob_not_rhs_given_lhs + eps)


### 可视化

In [12]:
rules_df['Lift'] = list(map(lambda x: lift(x), rules))
rules_df['Conviction'] = list(map(lambda x: conviction(x), rules))
rules_df

,Rules,Support,Confidence,Lift,Conviction
0,{US} -> {Cabernet Sauvignon},0.056289,0.134229,1.840278,1.070790
1,{Cabernet Sauvignon} -> {US},0.056289,0.771730,1.840278,2.543664
2,{Pinot Noir} -> {California},0.053081,0.518644,1.859703,1.498087
3,{California} -> {Pinot Noir},0.053081,0.190333,1.859703,1.108669
4,{US} -> {California},0.278885,0.665034,2.384614,2.152792
5,{California} -> {US},0.278885,1.000000,2.384614,580644.913096
6,{Virginie Boone} -> {California},0.073339,0.999476,3.583824,1372.835652
7,{California} -> {Virginie Boone},0.073339,0.262973,3.583824,1.257242
8,{US} -> {Chardonnay},0.052350,0.124835,1.380139,1.039287
9,{Chardonnay} -> {US},0.052350,0.578768,1.380139,1.378442


### 结果分析
根据上述的挖掘结果，我们可以挖掘出一些关于葡萄酒品种（variety）和原产地（country）、品酒师（taster_name）和原产地（country）以及省份（Province）和原产地（country）的关系，我们可以做出这样的假设：
1. 根据`{Cabernet Sauvignon} -> {US}`和`{US} -> {Cabernet Sauvignon}`两个规则，我们可以认为Cabernet Sauvignon很有可能大部分产自美国，且是美国产量占比最高的葡萄酒。
2. 根据`{US} -> {California}`和`{California} -> {US}`两个规则，我们知道California是美国的省份。
3. 根据`{Virginie Boone} -> {California}`和`{California} -> {Virginie Boone}`两个规则，我们可以认为Boone这个品酒师经常品鉴加利福尼亚州的葡萄酒。